<a href="https://colab.research.google.com/github/lilstarhunter/shrew_emesis/blob/main/shrew_emesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check that Runtime is set t GPU
# Install Dependencies
!pip install deeplabcut

     |████████████████████████████████| 541 kB 5.4 MB/s 
     |████████████████████████████████| 9.8 MB 18.5 MB/s 
     |████████████████████████████████| 296 kB 39.4 MB/s 
     |████████████████████████████████| 352 kB 36.1 MB/s 
     |████████████████████████████████| 47.6 MB 36 kB/s 
     |████████████████████████████████| 29.2 MB 59 kB/s 
     |████████████████████████████████| 177 kB 51.0 MB/s 
     |████████████████████████████████| 109 kB 49.4 MB/s 
     |████████████████████████████████| 948 kB 46.3 MB/s 
     |████████████████████████████████| 546 kB 48.7 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=8434b09e58540d2925e8f4237b6d8e3e12d7f72678128ee2ba419d0167828ec4
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
     

In [2]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
# Link Google Drive files

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import deeplabcut

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [5]:
import os
os.environ["DLClight"]="True"

In [10]:
#  Assign project variables
ProjectFolderName = 'vomitting-LMS-2021-10-19'
VideoType = 'mp4' 
videofile_path = ['/content/drive/My Drive/Colab Notebooks/shrew_emesis/'+ProjectFolderName+'/videos/'] 
path_config_file = '/content/drive/My Drive/Colab Notebooks/shrew_emesis/'+ProjectFolderName+'/config.yaml'
path_config_file

'/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/config.yaml'

In [12]:
# Create a training dataset
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19  already exists!
/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1  already exists!
/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1/train  already exists!
/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1/test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([ 90,  86,  71,  17,  97, 134, 109,   7, 157,  23, 170,  85,  87,
           11, 174, 150, 171, 177,  82,   2, 132, 154,  99,  20, 116,  10,
          178,  94, 123, 136,  34,  74,  65, 166, 121,  64, 149, 124, 165,
           75, 179, 152,  46,  41,  45,  18, 169, 113,  62,  69,   1,  31,
           83, 100,  36,  40, 120,  73, 111,  89, 106, 107,  88, 137,  13,
           15,  58,  32, 156, 163, 105, 147,  43,  98,  39,  56,  28, 184,
          130, 128, 133, 151, 148,  79, 167, 126,  52, 115, 125,  84,  96,
          143,   3,  37,   6,  22, 141,  19, 146,  91, 160, 114,  68,  21,
           59,  81, 180, 175,  51, 135,  33,  80,  54, 164,  48, 138, 161,
          181, 182,  29, 168, 101, 118, 103, 183, 127, 145, 108,  53, 119,
          112, 144,  49,   9,  60, 153, 155,  50,  57, 159, 104,  38,  77,
           14,  61, 110, 172,  95,  72,  25,  55, 158,  93,  70, 173, 129,
            0,  30,   8,   4, 139, 142,  63, 176,  26,  27,  92, 102, 162,
          11

In [13]:
# Training - Iteration 0
shuffle = 1
deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=200, maxiters=1000, allow_growth=True)

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['snout', 'mandible', 'back', 'abdomen', 'tailbase'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 1,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19/vomitting_LMS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 

Selecting single-animal trainer
Batch Size is 1
Loading ImageNet-pretrained resnet_50
Max_iters overwritten as 1000
Display_iters overwritten as 100
Save_iters overwritten as 200
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/dlc-models/iteration-0/vomittingOct19-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 

iteration: 100 loss: 0.0630 lr: 0.005
iteration: 200 loss: 0.0199 lr: 0.005
iteration: 300 loss: 0.0167 lr: 0.005
iteration: 400 loss: 0.0140 lr: 0.005
iteration: 500 loss: 0.0126 lr: 0.005
iteration: 600 loss: 0.0118 lr: 0.005
iteration: 700 loss: 0.0107 lr: 0.005
iteration: 800 loss: 0.0101 lr: 0.005
iteration: 900 loss: 0.0101 lr: 0.005
iteration: 1000 loss: 0.0093 lr: 0.005


The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1365, in _do_call
    return fn(*args)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1350, in _run_fn
    target_list, run_metadata)
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/client/session.py", line 1443, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node fifo_queue_enqueue}}]]

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/core/train.py", line 82, in load_and_enqu

In [14]:
# Evaluate the model
shuffle=1
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0,10,20])

Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['snout', 'mandible', 'back', 'abdomen', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19/vomitting_LMS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 5,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'scoremap_dir': 'test',
 'shuffle': True,
 'snapshot_pre

Running  DLC_resnet50_vomittingOct19shuffle1_1000  with # of trainingiterations: 1000
Analyzing data...


185it [00:26,  7.07it/s]


Done and results stored for snapshot:  snapshot-1000
Results for 1000  training iterations: 95 1 train error: 8.11 pixels. Test error: 8.2  pixels.
With pcutoff of 0.6  train error: 5.92 pixels. Test error: 7.36 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|██████████| 185/185 [01:16<00:00,  2.42it/s]
Config:
{'all_joints': [[0], [1], [2], [3], [4]],
 'all_joints_names': ['snout', 'mandible', 'back', 'abdomen', 'tailbase'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_vomittingOct19/vomitting_LMS95shuffle1.mat',
 'dataset_type': 'imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'mirror': False,
 'net_type': 'resnet_50',
 'num_joints': 5,
 'optimizer': 'sgd',
 'pairwise_huber_loss': True,
 'pairwise_predict': False,
 'partaffinityfield_predict': False,
 'regularize': False,
 'score

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)
/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/evaluation-results/  already exists!
/content/drive/My Drive/Colab Notebooks/shrew_emesis/vomitting-LMS-2021-10-19/evaluation-results/iteration-0/vomittingOct19-trainset95shuffle1  already exists!
Analyzing data...


3it [00:00,  3.55it/s]


Saving plots...


100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


In [ ]:
# Analyze videos
VideoType = 'MP4'
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

Using snapshot-10000 for model /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/dlc-models/iteration-0/ShrewTrackOct6-trainset95shuffle1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos  already exists!
Loading  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4
Duration of video [s]:  72.07 , recorded with  29.97 fps!
Overall # of frames:  2160  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture


2163it [11:39,  3.09it/s]


Saving results in /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos...
Starting to analyze %  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos  already exists!
Loading  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4
Duration of video [s]:  78.08 , recorded with  29.97 fps!
Overall # of frames:  2340  found with (before cropping) frame dimensions:  1920 1080
Starting to extract posture


2346it [12:22,  3.16it/s]

Saving results in /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_ShrewTrackOct6shuffle1_10000'

In [ ]:
## Assess pose estimation (may need to add more data and re-train)
Specific_videofile = '/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4'
deeplabcut.create_video_with_all_detections(path_config_file, Specific_videofile)

In [ ]:
#Check and edit:
numAnimals = 1 #how many animals do you expect to find?
tracktype= 'ellipse' #box, skeleton, ellipse -- ellipse is recommended.

#Optional: imagine you tracked a point that is not useful for assembly, 
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):
bodypart= 'midchest, midhind'
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        shuffle=shuffle, track_method=tracktype, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
# deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        # shuffle=shuffle, track_method=tracktype, overwrite=True)

deeplabcut.stitch_tracklets(path_config_file, videofile_path, track_method=tracktype, n_tracks=numAnimals)

In [ ]:
# filter the data to remove any small jitter
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType)
                                #  track_method = tracktype)

Analyzing all the videos in the directory...
Filtering with median model /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4
Saving filtered csv poses!
Filtering with median model /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4
Saving filtered csv poses!


In [ ]:
#Create plot trajectory
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4 and data.
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/plot-poses/MVI_0444  already exists!
Loading  /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4 and data.
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/plot-poses/MVI_0445  already exists!
Plots created! Please check the directory "plot-poses" within the video directory


In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                # color_by="individual",
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True) 
                                # track_method = tracktype)

Analyzing all the videos in the directory...
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos  already exists!
/content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos  already exists!
Starting to process video: /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4
Starting to process video: /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4
Loading /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0445.MP4 and data.
Loading /content/drive/My Drive/ShrewTracking-LaurenStein-2021-10-06/videos/MVI_0444.MP4 and data.
Duration of video [s]: 78.08, recorded with 29.97 fps!
Overall # of frames: 2340 with cropped frame dimensions: 1920 1080
Generating frames and creating video.


  1%|          | 21/2340 [00:00<01:19, 29.28it/s]

Duration of video [s]: 72.07, recorded with 29.97 fps!
Overall # of frames: 2160 with cropped frame dimensions: 1920 1080
Generating frames and creating video.


100%|██████████| 2340/2340 [02:22<00:00, 16.46it/s]
